In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

import sys
sys.path.append('/home/cat/code/gerbil/utils/')

#
import os
import numpy as np

#
import scipy

#
from visualize import Visualize
from track import Track
from track import filter_trace


Autosaving every 180 seconds


In [2]:
#####################################################
#####################################################
#####################################################

def find_behaviors(track,
                   threshold = 0.2,   # probability threshold
                   duration = 1.0,     # duration in seconds
                   fps = 25,
                   threshold_abs = 0.02,
                   ):
    
    ''' Function that searches for N frames in a row above some threshold
    
    '''

    # smooth probabilities
    fc = 1 # 1 hz filter applied
    track = filter_trace(track,
                           fc)



    # find bouts    
    idx = np.where(track>threshold)[0]
    
    # find bouts above threshold
    bouts = []
    start = None
    if track[0]>threshold:
        start = 0
    #
    for k in range(1,track.shape[0],1):

        # within or starting a bout
        if track[k]>=threshold:
            if start == None:
                start = k
        # outside or exiting a bout
        else:
            if start!= None:  # i.e. within a bout laready
                
                if (k-start)>=(duration*fps):
                    
                    # go backwards in time until probability is < abs_min
                    for p in range(k,start,-1):
                        if track[p]<threshold_abs:
                            start = p
                            break
                    
                    bouts.append(np.arange(start,k,1))

                #    
                start = None
    bouts = np.hstack(bouts)
    

    
    behaviors = track.copy()*0 + np.nan
    
    behaviors[bouts] = 1
    
    return behaviors
    
    

In [6]:
#####################################################
#####################################################
#####################################################

################## FIND APPROACH BEHAVIORS #################
#fname = '/media/cat/256GB/dan/simba/simon_classifier/2020_07_20_10_38_20_043555.npy'
fname = '/media/cat/256GB/dan/simba/2020_07_20_10_38_20_043555.npy'
fname_video = fname.replace('.npy', '.mp4')
fname_video_out_tag = 'annotated'

# run track through behavior detection algorithm
track = np.load(fname)
threshold = 0.15
duration = 0.25

#
behaviors = find_behaviors(track,
                          threshold,
                          duration)


################### MAKE VIDEO WITH ANNOTATIONS ################
v = Visualize()

# time to tag
start = 0           # video start
end = None         # video end
scale = 4           # downsample video for faster processing

#
tag_loc = [600,100]

#
v.make_annotated_video_single_track(behaviors,
                                    fname_video,
                                    fname_video_out_tag,
                                    tag_loc,
                                    start,
                                    end,
                                    scale)

#

width, height:  900 700


100%|██████████| 28802/28802 [01:03<00:00, 455.33it/s]


In [7]:
############

data = np.load(fname)
print (data.shape)

#
thresh = 0.15

#
fc = 1 # 1 hz filter applied
data = filter_trace(data,
                   fc)

#
t = np.arange(data.shape[0])/25

#
plt.figure()
plt.plot(t, data)
plt.xlabel("Time (sec)")
plt.ylabel("Approach likelihood")

plt.plot([t[0], t[-1]],
         [thresh, thresh],
         '--',
         label='approach threshold'
        )

idx = np.where(data>thresh)[0]

plt.scatter(t[idx],
         np.zeros(idx.shape[0])+.5,
        c='red',
            label='detected approaches'
        )

plt.xlim(t[0],t[-1])
plt.ylim(bottom=0)
plt.suptitle(fname)
plt.legend()
plt.show()

(28802,)
